This is a notebook to pull down all the information for figure 5, and figure S2 - S5 (ncbi hits). The design is to compair between each sample (barcode) instead of each flowcell as they sequenced the same thing. Therefore I just pulled all the ncbi hits for each flowcell for each barcode and merged them together according to the barcode.

In [2]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from itertools import product
import argparse
from ete3 import NCBITaxa 


In [3]:
ncbi = NCBITaxa()

In [4]:
# All information are in the BASEDIR for each replicate. Here again used replicate4 as an example
BASEDIR = '/home/yiheng/data/20170617_replicate4'

# here we define the folder name of the dataframe it created by capturing the folder from the BASDIR
folder_name = os.path.basename(BASEDIR)
column_name = folder_name.split('_')[-1]

In [5]:
# first check if the analysis folder is there
folder_list = 'analysis  basecalled_data  scripts  tracking  workspace'.split(' ')
for x in range(0,folder_list.count('')):
    folder_list.remove('')

if not set(os.listdir(os.path.abspath(BASEDIR))) == set (folder_list):
    print("Something wrong with basefolder. check it please.")

In [6]:
# get the dataframe there
dataframe = os.path.join(BASEDIR, 'analysis', 'summary_df_%s.tab' % folder_name)
sum_df = pd.read_csv(dataframe, sep='\t')

/home/yiheng/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
sum_df.columns

Index(['read_id', 'passes_filtering', 'sequence_length_template',
       'mean_qscore_template', 'barcode_arrangement', 'barcode_score', 'kit',
       'variant', 'pc_survived', 'nl_survived', 'qseqid_rg', 'sseqid_rg',
       'evalue_rg', 'length_rg', 'pident_rg', 'nident_rg', 'sacc_rg',
       'staxids_rg', 'scomnames_rg', 'read_length_pc_x', 'qseqid_nt',
       'sseqid_nt', 'evalue_nt', 'length_nt', 'pident_nt', 'nident_nt',
       'sacc_nt', 'staxids_nt', 'scomnames_nt', 'read_length_pc_y'],
      dtype='object')

In [8]:
# fills the nan with T/F for easier handling
sum_df.sseqid_rg.fillna(False, inplace=True)
sum_df.sseqid_nt.fillna(False, inplace=True)
# filter out the rg blast hit
total_reads = sum_df[((sum_df.sseqid_rg != False) | (sum_df.sseqid_nt != False)) & (sum_df.passes_filtering == True) & (sum_df.pc_survived == True) & (sum_df.nl_survived == True)]
ntblasthit_reads = total_reads[(total_reads.sseqid_rg == False) & (total_reads.sseqid_nt != False)]
pd.set_option('display.max_columns', None)

#Removing unncessary columns from joint_NCBI_df_taxonomy
#I hard coded to select as it depends on what information you need
for columns in ['passes_filtering',
       'mean_qscore_template', 'barcode_score', 'kit',
       'variant', 'pc_survived', 'nl_survived', 'qseqid_rg', 
       'evalue_rg', 'length_rg', 'pident_rg', 'nident_rg', 'sacc_rg',
       'staxids_rg', 'scomnames_rg', 'read_length_pc_x', 'qseqid_nt',
       'sseqid_nt', 'evalue_nt', 'length_nt', 'pident_nt', 'nident_nt',
       'sacc_nt','scomnames_nt', 'read_length_pc_y']:
    del total_reads[columns] 

In [9]:
###########This block for barcodes order
# now everything left is what we want to plot out
# arrange different barcodes
# REMEMBER: This need to be manual check:
# barcode01 is Pst infected sample, barcode02 is Zymo sample, 03 is nill sample, 04 is co-infection sample and 05 is Pyre sample
new_barcodes = ['barcode01', 'barcode02', 'barcode03', 'barcode04', 'barcode05']
if column_name == 'replicate1':
    total_reads.drop(total_reads.index[total_reads.barcode_arrangement.str.contains('barcode01')], inplace = True)
    total_reads.barcode_arrangement = total_reads.barcode_arrangement.replace(to_replace='barcode02', value='barcode01')
    total_reads.barcode_arrangement = total_reads.barcode_arrangement.replace(to_replace='barcode06', value='barcode02')
    total_reads.barcode_arrangement = total_reads.barcode_arrangement.replace(to_replace='barcode04', value='barcode06')
    total_reads.barcode_arrangement = total_reads.barcode_arrangement.replace(to_replace='barcode05', value='barcode04')
    total_reads.barcode_arrangement = total_reads.barcode_arrangement.replace(to_replace='barcode06', value='barcode05')

elif column_name == 'replicate2':
    # first delete the rows that contains the barcode we are going to replace
    # I think maybe better to first adjust the df to get rid of all the mis/unclassified reads so the df get smaller
    # and we do not need those reads for plotting
    
    total_reads.drop(total_reads.index[total_reads.barcode_arrangement.str.contains('barcode01')], inplace = True)
    total_reads.drop(total_reads.index[total_reads.barcode_arrangement.str.contains('barcode02')], inplace = True)
    total_reads.drop(total_reads.index[total_reads.barcode_arrangement.str.contains('barcode03')], inplace = True)
    total_reads.drop(total_reads.index[total_reads.barcode_arrangement.str.contains('barcode04')], inplace = True)
    total_reads.drop(total_reads.index[total_reads.barcode_arrangement.str.contains('barcode05')], inplace = True)
    total_reads.drop(total_reads.index[total_reads.barcode_arrangement.str.contains('unclassified')], inplace = True)
    total_reads.barcode_arrangement = total_reads.barcode_arrangement.replace(to_replace='barcode07', value='barcode01')
    total_reads.barcode_arrangement = total_reads.barcode_arrangement.replace(to_replace='barcode08', value='barcode02')
    total_reads.barcode_arrangement = total_reads.barcode_arrangement.replace(to_replace='barcode09', value='barcode05')
    total_reads.barcode_arrangement = total_reads.barcode_arrangement.replace(to_replace='barcode10', value='barcode04')
    total_reads.barcode_arrangement = total_reads.barcode_arrangement.replace(to_replace='barcode11', value='barcode03')
    
elif column_name == 'replicate3':
    total_reads.drop(total_reads.index[total_reads.barcode_arrangement.str.contains('barcode06')], inplace = True)
    total_reads.barcode_arrangement = total_reads.barcode_arrangement.replace(to_replace='barcode03', value='barcode06')
    total_reads.barcode_arrangement = total_reads.barcode_arrangement.replace(to_replace='barcode05', value='barcode03')
    total_reads.barcode_arrangement = total_reads.barcode_arrangement.replace(to_replace='barcode06', value='barcode05')

elif column_name == 'replicate4':
    total_reads.drop(total_reads.index[total_reads.barcode_arrangement.str.contains('barcode06')], inplace = True)
    total_reads.barcode_arrangement = total_reads.barcode_arrangement.replace(to_replace='barcode03', value='barcode06')
    total_reads.barcode_arrangement = total_reads.barcode_arrangement.replace(to_replace='barcode05', value='barcode03')
    total_reads.barcode_arrangement = total_reads.barcode_arrangement.replace(to_replace='barcode06', value='barcode05')
else:
    pass



/home/yiheng/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/yiheng/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [10]:
# now just filter out all the reads that are not going to used for ploting
barcode01_total = total_reads[total_reads.barcode_arrangement.str.contains('barcode01')]
barcode02_total = total_reads[total_reads.barcode_arrangement.str.contains('barcode02')]
barcode03_total = total_reads[total_reads.barcode_arrangement.str.contains('barcode03')]
barcode04_total = total_reads[total_reads.barcode_arrangement.str.contains('barcode04')]
barcode05_total = total_reads[total_reads.barcode_arrangement.str.contains('barcode05')]

# now concat them together. but remember that the index has to change to add the column for ncbi taxa
total_reads_filtered_barcodes = pd.concat([barcode01_total, 
                                           barcode02_total, 
                                           barcode03_total, 
                                           barcode04_total, 
                                           barcode05_total], ignore_index=True)

In [11]:
total_reads_filtered_barcodes.staxids_nt.fillna(False, inplace=True)
ntblasthit_reads_filtered_barcodes = total_reads_filtered_barcodes[(total_reads_filtered_barcodes.staxids_nt != False)]

In [12]:
ntblasthit_reads_filtered_barcodes = ntblasthit_reads_filtered_barcodes.reset_index(drop=True)

In [13]:
# as the concate before will result the staxid into a string
# so now change the string back to float so it can be recognized by the NCBITaxa
for taxid in ntblasthit_reads_filtered_barcodes['staxids_nt']:
    if ';' in str(taxid):
        taxid = taxid.split(';')[0]
    else:
        pass
    float(taxid)

In [14]:
#Function to generate taxonomy columns based on NCBITaxa results for NCBI hit dataframe, 
def search_rank_output_name_append_column(df, staxid_column, rank_search):
    """Input df, staxid_column from same df and rank_search (a desired taxonomic rank 
    from each staxid's lineage), outputs taxonomic name corresponding to rank_search or 'Unclassified' if
    unavailable and appends to df row by row"""
    rank_list = []
    for read_index in range(0, len(staxid_column)):
        taxid = ''
        if ';' in str(staxid_column[read_index]):
            taxid = staxid_column[read_index].split(';')[0]
        else:
            taxid = staxid_column[read_index]
        
        taxid_lineage = ''
        taxid_lineage = ncbi.get_lineage(taxid)
        
        names = ''
        names = ncbi.get_taxid_translator(taxid_lineage)
        
        ranks = ''
        ranks = ncbi.get_rank(taxid_lineage) #Dict
        
        ranks2names = ''
        ranks2names = {ranks[k]:names[k] for k in names.keys() & ranks}
        
        if rank_search in ranks2names.keys():
            rank_list.append(ranks2names[rank_search])#if rank in dict, print name
        else:
            rank_list.append('Unclassified')
    df[rank_search] = rank_list
# NOTE: Appending is always slow, try and find a better way e.g df.apply to a column based on staxids column


In [15]:
#ntblasthit_reads_filtered_barcodes_added_TaxaRank = ntblasthit_reads_filtered_barcodes.copy()
rank_list = ['superkingdom', 'phylum', 'class', 'order', 'family', 'genus', 'species']
for rank in rank_list:
    search_rank_output_name_append_column(ntblasthit_reads_filtered_barcodes, 
                                          ntblasthit_reads_filtered_barcodes.staxids_nt, 
                                          rank)

In [16]:
# add the flowcell column just in case
ntblasthit_reads_filtered_barcodes = ntblasthit_reads_filtered_barcodes.assign(Flowcell = column_name)
total_reads_filtered_barcodes = total_reads_filtered_barcodes.assign(Flowcell = column_name)

In [18]:
total_reads_filtered_barcodes.to_csv(r'/home/yiheng/analysis/WGS/%s_totaltaxa.tab' % column_name, header=column_name, index=None, sep='\t')
ntblasthit_reads_filtered_barcodes.to_csv(r'/home/yiheng/analysis/WGS/%s_nttaxa.tab' % column_name, header=column_name, index=None, sep='\t')